In [ ]:
# 코드 실행 전 seed를 지정하겠습니다.
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# local 작업으로 진행
# 판다스 추가 파라미터 사용 (usecols = 필요한 컬럼만 데이터로드)
true = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS-02/Section_4/ds-section4-sprint2/ai_sc42x_true.csv', encoding='utf-8', usecols=['text'])
fake = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS-02/Section_4/ds-section4-sprint2/ai_sc42x_fake.csv', encoding='utf-8', usecols=['text'])

# 타겟값 라벨링
true['label'] = 0
fake['label'] = 1

# 데이터프레임 병합
df = pd.concat([true, fake])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44898 non-null  object
 1   label   44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [ ]:
df.shape

(44898, 2)

In [ ]:
# 결측치 확인
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
# 중복값 제거
df  = df.drop_duplicates(['text'], keep='last').reset_index(drop=True)
df.shape

(38646, 2)

In [ ]:
# 데이터프레임 행을 기준으로 섞기
df = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df.head()

,text,label
0,The 2014 IRS filings for the Clinton Foundatio...,1
1,A Tennessee Trump supporter is ecstatic that O...,1
2,21st Century Wire says The mainstream media co...,1
3,This is what happens when racists don t get th...,1
4,PARIS (Reuters) - France said on Wednesday it ...,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [ ]:
# text를 Vectorizer한다.

tfidf = TfidfVectorizer(stop_words= 'english', max_features= 3000) # 불용어는 영어, 최대 특성은 3000개로 제한한다.

dtm = tfidf.fit_transform(df['text']) # 핏 이후 데이터타입은 sparse matrix of type가 된다.
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names()) # .todense 차원행렬로 변경하는 메소드
dtm

,000,10,100,11,12,13,14,15,150,16,17,18,19,20,200,2000,2001,2003,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,21,21st,21wire,22,23,24,...,wonder,word,words,work,worked,worker,workers,working,works,world,worried,worry,worse,worst,worth,wouldn,wounded,write,writing,written,wrong,wrongdoing,wrote,www,xi,year,years,yemen,yes,yesterday,york,young,youth,youtube,zealand,zero,zika,zimbabwe,zone,zuma
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.097196,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.171874,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.051650,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.057525,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.040123,0.0,0.000000,0.0,0.000000,0.0,0.132737,0.071756,0.0,0.00000,0.0,...,0.0,0.0,0.000000,0.038856,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.080698,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.059147,0.0,0.075487,0.0,0.000000,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.037892,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.067585,0.0,0.054408,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.058804,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.069024,0.052230,0.0,0.0,0.0,0.000000,0.0,0.000000,0.086143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38641,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.073378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
38642,0.043616,0.0,0.0,0.0,0.056434,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.048846,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.031587,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
38643,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.029136,0.029261,0.02881,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077282,0.0,0.000

In [ ]:
from sklearn.neighbors import NearestNeighbors

# KNN 모델랑 
nn = NearestNeighbors(n_neighbors=5, algorithm= 'kd_tree') # 가장 거리가 가까운 5개를 찾아야 하므로 5로 설정한다.
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
# distance에는 기준 인덱스와 유사한 5개의 유사도 확률, result에는 유사한 5개의 인덱스 값이 표시된다.
distance, result = nn.kneighbors([dtm.iloc[42].values])

# for문을 사용해 데이터의 text 문장과 인덱스를 출력한다.
for idx in result:
    print(df['text'][idx])

42       SAN FRANCISCO (Reuters) - A U.S. judge on Tues...
6802     SAN FRANCISCO (Reuters) - The U.S. Justice Dep...
31829    SAN FRANCISCO (Reuters) - A California federal...
23569    (Reuters) - A U.S. judge on Thursday refused t...
234      SAN FRANCISCO (Reuters) - The U.S. Justice Dep...
Name: text, dtype: object


In [ ]:
print('index_42 =', df.loc[42]['label'])
print('index_6802 =', df.loc[6802]['label'])
print('index_31829 =', df.loc[31829]['label'])
print('index_23569 =', df.loc[23569]['label'])
print('index_234 =', df.loc[234]['label'])

index_42 = 0
index_6802 = 0
index_31829 = 0
index_23569 = 0
index_234 = 0


In [ ]:
# 데이터프레임 복사
df0 = df.copy()

In [ ]:
# 대소문자 변경 + 정규표현식

import re

for idx, text in enumerate(df0['text']):
  text = text.lower() ####Your Code Here####
  text = text.replace('\n',' ')
  text = re.sub(r'[^a-z0-9 ]', '', text) ####Your Code Here####
  df0['text'][idx] = text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
# 결측치 확인

print(df0.shape)
print(df0.isnull().sum())

(38646, 2)
text     0
label    0
dtype: int64


In [ ]:
# 베이스모델 확인

df['label'].value_counts(normalize=True)

0    0.548336
1    0.451664
Name: label, dtype: float64

In [ ]:
# 훈련데이터, 테스트데이터 분할하기
train, test = train_test_split(df0, test_size= 0.2, random_state= 42)
train.shape, test.shape

((30916, 2), (7730, 2))

In [ ]:
# 특성값, 타겟값 지정
x_train = train['text']
y_train = train['label']

x_test = test['text']
y_test = test['label']

print(f'Train : x_train= {x_train.shape}, y_train= {y_train.shape}')
print(f'Test : x_test= {x_test.shape}, y_test= {y_test.shape}')

Train : x_train= (30916,), y_train= (30916,)
Test : x_test= (7730,), y_test= (7730,)


In [ ]:
# 구글 뉴스 데이터 임베딩 다운
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
'''
# 케라스 Tokenizer(파라미터) : 
- num_words = 빈도에 따라 사용할 단어 개수의 최대값
- filter = 텍스트에서 제외할 문자, 정규표현식과 다르며 '!"#$%&()*+,-./:;<=>?' 와 같이 입력
- lower = True / False, 텍스트를 소문자로 변경할지 여부 
- split = ' ' / 따옴표 사이에 분해 기준이 되는 문자열 입력
- char_level = True / False, 문자를 토큰처리 할지 여부
''';

In [ ]:

# 토큰화 하기

tokenizer = Tokenizer(num_words= 1000)
tokenizer.fit_on_texts(x_train)

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

197979


In [ ]:
# .texts_to_sequences = 문자열을 인덱스의 리스트로 변환한다.
X_train_encoded = tokenizer.texts_to_sequences(x_train)
X_test_encoded = tokenizer.texts_to_sequences(x_test)

# 가장 긴 시퀸스와 평균 시퀸스를 알아본다.
train_max_len = max(len(i) for i in X_train_encoded)
train_mean_len = np.mean([len(i) for i in X_train_encoded])
print(f'Train : Max_Sequence = {train_max_len}, Mean_Sequence = {train_mean_len:.4f}')

test_max_len = max(len(i) for i in X_test_encoded)
test_mean_len = np.mean([len(i) for i in X_test_encoded])
print(f'Test : Max_Sequence = {test_max_len}, Mean_Sequence = {test_mean_len:.4f}')

Train : Max_Sequence = 5087, Mean_Sequence = 284.9839
Test : Max_Sequence = 4317, Mean_Sequence = 289.7467


In [ ]:
# pad_sequence
## 최대 길이는 평균 시퀸스 길이보다 조금 긴 300으로 지정

x_train_ps = pad_sequences(X_train_encoded, maxlen= 300)
x_test_ps = pad_sequences(X_test_encoded, maxlen= 300)

x_train_ps.shape, x_test_ps.shape

((30916, 300), (7730, 300))

In [ ]:
# 임베딩 가중치 행렬

embedding_matrix = np.zeros((vocab_size, 300))

print(np.shape(embedding_matrix))

(197979, 300)


In [ ]:
# 해당 단어가 word2vec에 있을 경우 임베딩 벡터를 반환하는 함수

def get_vector(word):
  if word in wv: 
    return wv[word]
  else:
    return None
 
for word, i in tokenizer.word_index.items(): # 토큰화된 x_train의 dict 데이터를 key(word), value(index) 형태로 하나씩 꺼낸다.
  temp = get_vector(word)                  # word를 wv와 대조하여 임베딩 벡터를 얻고
  if temp is not None:                     # 없는 단어라서 임베딩 벡터가 없는 경우 
    embedding_matrix[i] = temp           

In [ ]:
# 신경망 구축
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False))
model.add(GlobalAveragePooling1D()) # 입력되는 단어 벡터의 평균을 구한다.
model.add(Dense(1, activation='relu'))

In [ ]:
# 학습 실행

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x_train_ps, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
387/387 [==============================] - 7s 17ms/step - loss: 1.7625 - acc: 0.5497 - val_loss: 0.7120 - val_acc: 0.5437
Epoch 2/10
387/387 [==============================] - 6s 16ms/step - loss: 0.6695 - acc: 0.6520 - val_loss: 0.6491 - val_acc: 0.6750
Epoch 3/10
387/387 [==============================] - 6s 15ms/step - loss: 0.6242 - acc: 0.6997 - val_loss: 0.6080 - val_acc: 0.7107
Epoch 4/10
387/387 [==============================] - 6s 15ms/step - loss: 0.5821 - acc: 0.7486 - val_loss: 0.5639 - val_acc: 0.7633
Epoch 5/10
387/387 [==============================] - 6s 15ms/step - loss: 0.5371 - acc: 0.7824 - val_loss: 0.5177 - val_acc: 0.7874
Epoch 6/10
387/387 [==============================] - 6s 15ms/step - loss: 0.4920 - acc: 0.8029 - val_loss: 0.4720 - val_acc: 0.8037
Epoch 7/10
387/387 [==============================] - 6s 15ms/step - loss: 0.4501 - acc: 0.8149 - val_loss: 0.4315 - val_acc: 0.8169
Epoch 8/10
387/387 [==============================] - 6s 15ms/step - 

In [ ]:
model.evaluate(x_test_ps, y_test)

242/242 [==============================] - 1s 6ms/step - loss: 0.3635 - acc: 0.8406


[0.3634934425354004, 0.8406209349632263]

In [ ]:
from tensorflow.keras.models import Sequential

model1 = Sequential()
model1.add(Embedding(vocab_size, 64))
model1.add(LSTM(32, dropout= 0.2, recurrent_dropout= 0.2))
model1.add(Dense(1, activation= 'relu'))

In [ ]:
# 학습 실행

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          12670656  
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 12,683,105
Trainable params: 12,683,105
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.fit(x_train_ps, y_train, batch_size=64, epochs=2, validation_split=0.2)

Epoch 1/2
387/387 [==============================] - 201s 512ms/step - loss: 0.4097 - acc: 0.8387 - val_loss: 0.2083 - val_acc: 0.9465
Epoch 2/2
387/387 [==============================] - 196s 507ms/step - loss: 0.2026 - acc: 0.9548 - val_loss: 0.1295 - val_acc: 0.9757


In [ ]:
model1.evaluate(x_test_ps, y_test)

242/242 [==============================] - 8s 35ms/step - loss: 0.1182 - acc: 0.9728


[0.11822047829627991, 0.9728330969810486]